In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split
from keras import models
from keras import layers
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
%matplotlib inline


Using TensorFlow backend.
/Users/thomascurran/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
test = pd.read_csv("/Users/thomascurran/desktop/MACS30200proj2/Data/mstest2_withlettergrade.csv").drop("Unnamed: 0", axis = 1)


In [9]:
test['Year'] = test['Year'].astype('category')
test.drop('IndexScore', axis = 1, inplace = True)

In [10]:
#test2 = test.select_dtypes(include = ['int', 'float'])

In [11]:
y = test['LetterGrade']
x = test.select_dtypes(include = ['int', 'float'])
xtrain, xtest, ytrain, ytest = train_test_split(x,y, train_size = .65, random_state = 25)

In [12]:
from sklearn.preprocessing import normalize, scale

'''
Parameters:	
X : {array-like, sparse matrix}

The data to center and scale.

axis : int (0 by default)

axis used to compute the means and standard deviations along. If 0, independently standardize each feature, otherwise (if 1) standardize each sample.

with_mean : boolean, True by default

If True, center the data before scaling.

with_std : boolean, True by default

If True, scale the data to unit variance (or equivalently, unit standard deviation).

copy : boolean, optional, default True

set to False to perform inplace row normalization and avoid a copy (if the input is already a numpy array or a scipy.sparse CSC matrix and if axis is 1).


'''

xtrain2 = xtrain.copy()
for i in range(0, len(xtrain.columns)):
    a = scale(xtrain2.iloc[:,i], with_mean = True, with_std = True)
    xtrain2.iloc[:,i] = a
   
    

In [18]:
#xtrain2 = xtrain2.values
ytrain2 = ytrain.copy().values

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(ytrain2)
encoded_Y = encoder.transform(ytrain2)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [48]:
network = models.Sequential()

network.add(layers.Dense(512, activation = 'relu', input_shape = (xtrain2.shape[1],))) 
network.add(layers.Dense(512, activation = 'relu', input_shape = (xtrain2.shape[1],))) 
network.add(layers.Dense(512, activation = 'relu', input_shape = (xtrain2.shape[1],))) 
network.add(layers.Dense(512, activation = 'relu', input_shape = (xtrain2.shape[1],))) 
network.add(layers.Dense(5, activation = "softmax"))

network.compile(optimizer = 'rmsprop',
                loss = 'categorical_crossentropy',
                metrics = ['accuracy']
               )

network_training_1 = network.fit(xtrain2,
                                 dummy_y, 
                                 epochs = 200,
                                 validation_split = len(xtrain2) * .75,
                                 batch_size = 512,
                                 verbose = 1)

test_loss, test_acc = network.evaluate(xtrain2, dummy_y)
print("Test Loss: ", test_loss)
print("Test Accuracy: ", test_acc)

Epoch 1/200
577/577 [==============================] - 0s 841us/step - loss: 1.6025 - acc: 0.2478
Epoch 2/200
577/577 [==============================] - 0s 101us/step - loss: 1.4506 - acc: 0.4021
Epoch 3/200
577/577 [==============================] - 0s 123us/step - loss: 1.2553 - acc: 0.4645
Epoch 4/200
577/577 [==============================] - 0s 115us/step - loss: 1.3812 - acc: 0.3847
Epoch 5/200
577/577 [==============================] - 0s 113us/step - loss: 1.1454 - acc: 0.4974
Epoch 6/200
577/577 [==============================] - 0s 109us/step - loss: 1.1101 - acc: 0.5286

In [53]:
def baseline():
    #create model:
    model = Sequential()
    model.add(layers.Dense(100, 
                           activation = 'relu',
                           input_shape = (xtrain2.shape[1],))
             ) 
    model.add(layers.Dense(5, activation = "softmax"))
    
    #compile model:
    model.compile(loss = "categorical_crossentropy",
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    return model

estimator = KerasClassifier(build_fn = baseline, epochs = 200, batch_size = 5, verbose = 1)

kfold = KFold(n_splits = 10, shuffle = True, random_state = 1234)

results = cross_val_score(estimator, xtrain, dummy_y, cv = kfold)

print(results.mean())

Epoch 1/200
519/519 [==============================] - 0s 656us/step - loss: 11.8013 - acc: 0.2678
Epoch 2/200
519/519 [==============================] - 0s 311us/step - loss: 11.8013 - acc: 0.2678
Epoch 3/200
519/519 [==============================] - 0s 319us/step - loss: 11.8013 - acc: 0.2678
Epoch 4/200
519/519 [==============================] - 0s 333us/step - loss: 11.8013 - acc: 0.2678
Epoch 5/200
519/519 [==============================] - 0s 308us/step - loss: 11.8013 - acc: 0.2678
Epoch 6/200
519/519 [==============================] - 0s 339us/step - loss: 11.8013 - acc: 0.2678

In [54]:
results.mean()

0.20644283532843902

In [ ]:
def model1():
    #create model:
    model = Sequential()
    model.add(layers.Dense(500, 
                           activation = 'relu',
                           input_shape = (xtrain2.shape[1],))
             ) 
    model.add(layers.Dense(500, 
                           activation = 'relu',
                           input_shape = (xtrain2.shape[1],))
             ) 
    model.add(layers.Dense(5, activation = "softmax"))
    
    #compile model:
    model.compile(loss = "categorical_crossentropy",
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    return model

estimator = KerasClassifier(build_fn = model1, epochs = 200, batch_size = 5, verbose = 0)

kfold = KFold(n_splits = 10, shuffle = True, random_state = 1234)

results = cross_val_score(estimator, xtrain, dummy_y, cv = kfold)

print(results.mean())

In [ ]:
results.mean()